# RNN Based molucule generation 

Laurent Cetinsoy

In the hands-on we will train a character based RNN to generate smile molecules

We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model ! 

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [ ]:
!curl https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv -o smiles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.5M  100 21.5M    0     0  46.7M      0 --:--:-- --:--:-- --:--:-- 46.7M


Import pandas and load the first 1000 lines

In [ ]:
import pandas as pd

df_smiles = pd.read_csv("smiles.csv")
df_smiles = df_smiles.head(1000)
len(df_smiles)

1000

Display the first rows of the dataframe

In [ ]:
df_smiles.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [ ]:
max(df_smiles.smiles.str.len())

106


Code a function **unic_characters(string)** which return the unic characters in a string


In [ ]:
def unic_characters(s):
  return set(s)


Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [ ]:
token = unic_characters(df_smiles.smiles.str.cat(sep=None))
token

{'\n',
 '#',
 '(',
 ')',
 '+',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '=',
 '@',
 'B',
 'C',
 'F',
 'H',
 'I',
 'N',
 'O',
 'S',
 '[',
 '\\',
 ']',
 'c',
 'l',
 'n',
 'o',
 'r',
 's'}

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")


In [ ]:
def map_char_to_int(unic_chars):
  mapping = {}
  reverse_mapping = {}
  for i, char in enumerate(unic_chars):
    mapping[char] = i
    reverse_mapping[i] = char
  return mapping, reverse_mapping

mapping, reverse_mapping = map_char_to_int(token)
mapping

{'\n': 5,
 '#': 22,
 '(': 25,
 ')': 20,
 '+': 11,
 '-': 24,
 '/': 15,
 '1': 4,
 '2': 1,
 '3': 2,
 '4': 23,
 '5': 14,
 '6': 16,
 '7': 12,
 '=': 9,
 '@': 21,
 'B': 13,
 'C': 19,
 'F': 27,
 'H': 30,
 'I': 31,
 'N': 8,
 'O': 28,
 'S': 10,
 '[': 29,
 '\\': 7,
 ']': 17,
 'c': 0,
 'l': 32,
 'n': 3,
 'o': 18,
 'r': 6,
 's': 26}

Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [ ]:
reverse_mapping

{0: 'c',
 1: '2',
 2: '3',
 3: 'n',
 4: '1',
 5: '\n',
 6: 'r',
 7: '\\',
 8: 'N',
 9: '=',
 10: 'S',
 11: '+',
 12: '7',
 13: 'B',
 14: '5',
 15: '/',
 16: '6',
 17: ']',
 18: 'o',
 19: 'C',
 20: ')',
 21: '@',
 22: '#',
 23: '4',
 24: '-',
 25: '(',
 26: 's',
 27: 'F',
 28: 'O',
 29: '[',
 30: 'H',
 31: 'I',
 32: 'l'}

For each smile molecule add the ending token to it

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

In [ ]:
smiles = "CCc1NO(N(Cc2ccc(C)s2)C(=O)c2ccc(=O)n(C)n2)cc1\n"

X = [
     ["CCc1"],
     ["Cc1N"],
     ["c1NO"]
]

y = [
     "N",
     "O",
     "("
]

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [ ]:
def build_X_and_y(string, i_char, seq_length, mapping):

  i_stop = i_char + seq_length
  X = [mapping[char] for char in string[i_char: i_stop]]
  y= [mapping[char] for char in string[i_stop]]

  return X, y

X1, y1 = build_X_and_y(smiles, 0, 4, mapping)
print(X1, y1)

[19, 19, 0, 4] [8]


In [ ]:
def build_all_sequences(string, seq_length, mapping):

  X_data = []
  y_data = []
  i_max = len(string) - seq_length
  for i in range(i_max):
    X, y = build_X_and_y(string, i, seq_length, mapping)
    X_data.append(X)
    y_data.append(y)
  return X_data, y_data

X, y = build_all_sequences(smiles, 4, mapping)
X,y    

([[19, 19, 0, 4],
  [19, 0, 4, 8],
  [0, 4, 8, 28],
  [4, 8, 28, 25],
  [8, 28, 25, 8],
  [28, 25, 8, 25],
  [25, 8, 25, 19],
  [8, 25, 19, 0],
  [25, 19, 0, 1],
  [19, 0, 1, 0],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 25],
  [0, 0, 25, 19],
  [0, 25, 19, 20],
  [25, 19, 20, 26],
  [19, 20, 26, 1],
  [20, 26, 1, 20],
  [26, 1, 20, 19],
  [1, 20, 19, 25],
  [20, 19, 25, 9],
  [19, 25, 9, 28],
  [25, 9, 28, 20],
  [9, 28, 20, 0],
  [28, 20, 0, 1],
  [20, 0, 1, 0],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 25],
  [0, 0, 25, 9],
  [0, 25, 9, 28],
  [25, 9, 28, 20],
  [9, 28, 20, 3],
  [28, 20, 3, 25],
  [20, 3, 25, 19],
  [3, 25, 19, 20],
  [25, 19, 20, 3],
  [19, 20, 3, 1],
  [20, 3, 1, 20],
  [3, 1, 20, 0],
  [1, 20, 0, 0],
  [20, 0, 0, 4]],
 [[8],
  [28],
  [25],
  [8],
  [25],
  [19],
  [0],
  [1],
  [0],
  [0],
  [0],
  [25],
  [19],
  [20],
  [26],
  [1],
  [20],
  [19],
  [25],
  [9],
  [28],
  [20],
  [0],
  [1],
  [0],
  [0],
  [0],
  [25],
  [9],
  [28],
  [20],
  [3],
  [2

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

In [ ]:
X_train = []
y_train = []
seq_length = 5

for molecule in df_smiles.smiles:
  X, y = build_all_sequences(molecule, seq_length, mapping)
  X_train = X_train + X
  y_train = y_train + y

X_train[:3], y_train[:3]

([[19, 19, 25, 19, 20], [19, 25, 19, 20, 25], [25, 19, 20, 25, 19]],
 [[25], [19], [20]])

Create numpy arrays from the lists

In [ ]:
import numpy as np
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train[:3], y_train[:3]

(array([[19, 19, 25, 19, 20],
        [19, 25, 19, 20, 25],
        [25, 19, 20, 25, 19]]), array([[25],
        [19],
        [20]]))

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [ ]:
X_train = X_train.reshape(-1, seq_length, 1)

Normalize X by dividing each values by the total number of unic characters

In [ ]:
X_train = X_train / len(token)

In [ ]:
X_train[:1], y_train[:1]

(array([[[0.57575758],
         [0.57575758],
         [0.75757576],
         [0.57575758],
         [0.60606061]]]), array([[25]]))

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, LSTM, Flatten

In [ ]:
model = Sequential()
model.add(LSTM(30, activation="tanh", input_shape=(None, 1)))
model.add(Flatten())
model.add(Dense(800, activation="relu"))
model.add(Dense(len(token), activation="softmax"))

Compile the model with the appropriate loss function and the adam optimizer

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30)                3840      
_________________________________________________________________
flatten (Flatten)            (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 800)               24800     
_________________________________________________________________
dense_1 (Dense)              (None, 33)                26433     
Total params: 55,073
Trainable params: 55,073
Non-trainable params: 0
_________________________________________________________________


Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

In [ ]:
model.fit(X_train[:10], y_train[:10], epochs = 500)

Epoch 1/500
1/1 [==============================] - 0s 8ms/step - loss: 2.6623
Epoch 2/500
1/1 [==============================] - 0s 10ms/step - loss: 2.5801
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 2.4957
Epoch 4/500
1/1 [==============================] - 0s 11ms/step - loss: 2.4107
Epoch 5/500
1/1 [==============================] - 0s 10ms/step - loss: 2.3267
Epoch 6/500
1/1 [==============================] - 0s 9ms/step - loss: 2.2456
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 2.1690
Epoch 8/500
1/1 [==============================] - 0s 8ms/step - loss: 2.0986
Epoch 9/500
1/1 [==============================] - 0s 7ms/step - loss: 2.0357
Epoch 10/500
1/1 [==============================] - 0s 12ms/step - loss: 1.9808
Epoch 11/500
1/1 [==============================] - 0s 10ms/step - loss: 1.9343
Epoch 12/500
1/1 [==============================] - 0s 11ms/step - loss: 1.8961
Epoch 13/500
1/1 [==============================] - 0s 

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


In [ ]:
def make_prediction(seed_start, model, n_token):
  seq = seed_start
  for i in rnage(50):
    seq = seq.reshape(1, seq_lentgh, 1)
    next_token = model.predict_classes(seq)[0]
    str_token = reverse_mapping[next_token]
    print(next_token, str_token)
    seq[:4] = seq[:1]
    seq[4] = next_token

seed_start = np.random.randint(0, n_token, seq_length) / n_token  #normalization
print("initial sequence", [reverse_mapping[c] for c in seed_start * n_token])
make_prediction(seed_start, model, len(token))

generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

In [ ]:
model.fit(X_train, y_train, epochs = 10)

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : predict the molecule solubility of your generated molecules 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better